In [4]:

try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

# import logging
import matplotlib.pyplot as plt
import numpy as np
import os

# logging.basicConfig(format=
  #                        "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
 #                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
bpl.output_notebook()
#####

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [6]:
#Paths to raw and ffc. 
fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05"
fnames    = [os.path.join(fld, '19JUN05_BF164p1_v1_ds126_crop_ffcSub.tif')]  # ffc filename to be processed
fname_raw = [os.path.join(fld, '19JUN05_BF164p1_v1_ds126_crop_raw.tif')]  ## raw file

memmapped = os.path.join(fld, "19JUN05_BF164p1_v1_ds126_crop_ffcSub_els__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap")


In [7]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

## CNMF-E params

In [10]:
# dataset dependent parameters
frate = 10.                       # movie frame rate
decay_time = 2.              # length of a typical transient in seconds

#static parameters
#pw_rigid = True  # flag for performing rigid or piecewise rigid motion correction
border_nan = 'copy'  # replicate values along the boundary (if True, fill in with NaN)


opts = params.CNMFParams()


# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = 15  ##changed          # upper bound on number of components per patch, in general None
gSig = (2.5, 2.5)     #2.5  # gaussian width of a 2D gaussian kernel, which approximates a neuron. 
gSiz = (8, 8)     # average diameter of a neuron, in general 4*gSig+1. 
Ain = None          # possibility to seed with predetermined binary masks
merge_thresh = .6   # default 0.7. merging threshold, max correlation allowed
rf = 30             # default 40. half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 15     # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .8       # min peak value from correlation image
min_pnr = 6        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor
bord_px = 0
opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thresh': merge_thresh,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,  # whether to remove duplicates from initialization                                        
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

CNMFParams:

data:

{'caiman_version': '1.8.3',
 'decay_time': 0.4,
 'dims': None,
 'dxy': (1, 1),
 'fnames': None,
 'fr': 30,
 'last_commit': 'GITW-e27d38526c04109da59802cb04d7d43056b0fe05',
 'mmap_C': None,
 'mmap_F': None,
 'var_name_hdf5': 'mov'}

spatial_params:

{'block_size_spat': 5000,
 'dist': 3,
 'expandCore': array([[0, 0, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0],
       [0, 0, 1, 0, 0]]),
 'extract_cc': True,
 'maxthr': 0.1,
 'medw': None,
 'method_exp': 'dilate',
 'method_ls': 'lasso_lars',
 'n_pixels_per_process': None,
 'nb': 0,
 'normalize_yyt_one': True,
 'nrgthr': 0.9999,
 'num_blocks_per_run_spat': 20,
 'se': array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=uint8),
 'ss': array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=uint8),
 'thr_method': 'nrg',
 'update_background_components': True}

temporal_params:

{'ITER': 2,
 'bas_nonneg': False,
 'block_size_temp': 5000,
 'fudge_factor': 0.96,
 'lags': 5,
 'me

In [9]:
Yr, dims, T = cm.load_memmap(memmapped)

images = Yr.T.reshape((T,) + dims, order='F')

In [11]:
# Show the average background
f, axarr = plt.subplots(ncols=2, nrows=1, figsize=(10, 5))
axarr[0].imshow(images[1000, :, :], cmap="gray")
axarr[1].imshow(np.median(images, axis=0), cmap="gray")

In [7]:
# compute some summary images (correlation and peak to noisae)
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False) 

# inspect the summary images and set the parameters
#inspect_correlation_pnr(cn_filter, pnr)

### Fit CNMF-E

In [12]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)


Exception: The file is in F order, it should be in C order (see save_memmap function)